# chapter 3. CUDA C에 대한 소개

## 학습목표

* CUDA C 코드를 작성하는 방법을 배운다.
* host 코드와 device 코드의 차이점을 배운다.
* host에서 device 코드를 실행하는 방법을 배운다.
* CUDA-capable device에서 디바이스 메모리를 사용하는 방법을 배운다.
* CUDA 디바이스의 시스템 정보를 질의 하는 방법을 배운다.

## 환경설정 : for CUDA C  
출처 :https://harshityadav95.medium.com/how-to-run-cuda-c-or-c-on-google-colab-or-azure-notebook-ea75a23a5962

**step1** server의 CUDA cloud instance를 재시작한다.

In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

**step2** CUDA version 9을 설치한다.

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-02-25 07:46:17--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-02-25 07:46:18--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?ZUQ-unOnSkH5C-H_5HUrBxmyXDs6pD8Pq-g-TI1I9WPsxQqI2KOdKOV6tOOezK5l4Zu9vLF8IoPKuDblzGmwj97cjWonceEGyO240FDLTDIXqXh

**step3** CUDA version을 확인해보자.

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


**step4** Notebook에서 nvcc를 실행시키기 위해 small extension을 설치한다.

In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gylqpem7
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gylqpem7
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=54b56c9df4ce2a58d7f1a17d409fecaee9f99a6ca2bf7ce16353de8ba5a6cb2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-p9ym0z62/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


**step5** extension을 불러온다.

In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## 첫번째 실습 : HelloWorld!

In [9]:
%%cu
//code 1.1
#include <stdio.h>

int main(void) {
    printf("hello world!\n");
    return 0;
}

hello world!



위의 코드는 단순히 우리가 알고 있는 C언어에서의 "hello world!"를 출력해내기 위한 코드와 동일하다.   
이는 위의 예제가 완전히 **host**에서만 실행되기 때문에 그렇다.   

그렇다면, **device**에서 코드가 실행되기 위해서는 어떻게 해야할 까?
※host : CPU, device : GPU

### 커널 호출

In [ ]:
%%cu
//code 1.2
#include <stdio.h>

__global__ void  kernel(void) {

}

int main() {
    kernel<<<1, 1>>>();
    printf("hello world!\n");
    
    return 0;
}

hello world!



위의 코드에서는 조금 특이한 코드들을 확인할 수 있다.   
1. \_\_gobal\_\_ 수식어구가 붙은 kernel()이라는 이름의 비어있는 함수
2. 비어 있는 함수에 대한 \<\<\<1, 1\>\>\>으로 장식된 호출 

앞선 code 1.1의 코드는 기본으로 설정된 표준 C컴파일러로 컴파일이 가능하다.
하지만, 이는 **host**에서 작동하는 것이고 우리는 **device**에서 작동하는 코드를 공부할 것이다.   


#### 1. \_\_gobal\_\_ 수식어구가 붙은 kernel()이라는 이름의 비어있는 함수   

해당 메커니즘은 이 함수는 host가 아닌 device에서 작동되도록 컴파일 되어야한 다는 것을 compile에 알려준다.

#### 2. 비어 있는 함수에 대한 \<\<\<1, 1\>\>\>으로 장식된 호출 

* 런타임 시스템에 넘겨질 인자를 의미한다.
* 해당 인자들은 device 코드로 전달되지 않으나 런타임이 디바이스 코드를 **어떻게 launch**하는 지에 영향을 미치는 매개 변수들이다.

### 매개변수 전달

In [ ]:
%%cu
// code 1.3
#include <stdio.h>
static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))
__global__ void add(int a, int b, int* c) {
    *c = a + b;
}

int main(void) {
    int c;
    int *dev_c;

    HANDLE_ERROR( cudaMalloc((void**)&dev_c, sizeof(int)));

    add<<<1, 1>>>(2, 7, dev_c);

    HANDLE_ERROR( cudaMemcpy( &c,
                             dev_c,
                             sizeof(int),
                             cudaMemcpyDeviceToHost));
    printf("2 + 7 = %d\n", c);
    cudaFree(dev_c);

    return 0;
}

2 + 7 = 9



* CUDA C도 C함수 처럼 커널에 매개 변수를 전달할 수 있다.
* 반환 값을 host에 전달하는 것처럼, device에서 수행할 작업을 위해 메모리를 할당해야한다.

여기서는 kernel호출을 하는 것이 눈에 띄일 뿐, 별다른 것을 확인할 수가 없다.   
이는 host에서 device로 매개변수들이 전달될 때 복잡한 작업들이 필요로 하지만, 런타임 시스템이 이를 다루고 있기에 우리는 해당 과정에서는 그러한 작업을 볼 수 가 없다.

#### cudaMalloc()에 의한 메모리 할당.

해당 호출은 C언어의 malloc과 유사하다. 하지만, CUDA 런타임이 device 메모리를 할당한다는 점이 차이가 난다,
* 첫 번째, 파라미터는 새로 할당할 메모리의 주소를 가리키고 있는 포인터이다.  
* 두 번째, 파라미터는 할당할 메모리의 크기이다.
할당한 메모리를 가리키는 포인터는 함수의 반환값이 아니다.
해당 부분은 반환 타입이 void\*인 malloc과 차이가 있다. **하지만 그 기능은 동일하다.**

##### 주의 해야할 점 : host & device
CUDA C는 host와 device의 코드간 경계를 모호하게 함으로써 단순함과 강력함을 동시에 지니고 있다.   
여기서, host에서 실행되는 코드에서 cudaMalloc()에 의해 반환되는 포인터를 역참조하지 않아야 하는 것은 프로그래머의 책임이다.  
host 코드에서는 포인터의 위치를 이동하거나, 초인터를 이용하여 어떤 연산을 수행하는지, 또는 포인터를 어떤 타입 변환을 할수도 있다.    
**하지만** 메모리를 읽거나 기록하기 위해서는 사용할 수 없다.    

하지만, 이러한 문제들은 compiler에서 방지해주지 않는다.   
이는 application에 있는 device 포인터들도 다른 포인터들과 동일한 것처럼 취급되기 때문이다.   
즉, compiler는 host 코드에 있는 device 포인터들의 역참조를 완전히 허용한다.   

그렇다면, device 포인터사용에 대한 제약사항들을 정리해보자.    

* cudaMalloc()으로 할당한 메모리 포인터를 device에서 실행되는 함수로 전달할 수 있다.   
* device에서 실행되는 코드에서 cudaMalloc()으로 할당한 메모리 포인터를 이용하여 메모리를 읽거나 쓸수 있다.   
* cudaMalloc()으로 할당한 메모리 포인터를 host에서 실행되는 함수로 전달할 수 있다.   
* host에서 실행되는 코드에서 cudaMalloc()으로 할당한 메모리 포인터를 이용하여 **메모리를 읽거나 쓸 수 없다.**

#### cudaFreee()로 메모리 해제   

device 메모리를 할당하고 해제하기 위해 host에서 어떻게 하는지 code 1.3에서 알 수 있었다.  
```cpp   
cudaFree(dev_c);
```   
그리고 host에서는 이 메모리를 변경할 수 없다는 것을 알게 되었다.   

또한 device메모리에 접근하기 위한 가장 보편적인 두가지 방법이 존재한다.   
1. device code에서 device 포인터를 사용하는 것   
```cpp   
__global__ void add(int a, int b, int* c) {
    *c = a + b;
}
```   

2. cudaMemcpy()를 호출함으로써 메모리에 접근하는 것   
```cpp   
cudaMemcpy( &c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);
```


host에서 작동하는 표준 C포인터들을 사용하는 것과 같은 방식으로 device 코드에서 포인터들을 사용한다,   
```cpp   
*c = a + b   
```   

**host 포인터들은 host code에서 메모리에 접근할 수 있고, device 포인터들은 device code에서 메모리에 접근할 수 있다.**   

#### cudaMemcpy()로 메모리 복사하기   

host code에서는 cudaMemcpy()를 호출해서 device 메모리에 접근할 수 있다.   
```cpp   
cudaMemcpy( &c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);   
```   
여기서 우리는 `cudaMemcpyDeviceToHost`에 주목해보자. (나머지 파라미터의 경우, 표준 C에서 memcpy와 동일하기 때문에 넘어가도록 하자.)   

해당 파라미터는 source 포인터와 destination 포인터 중 어느 것이 device 포인터를 가리키는 지 나타낸다.   

* `cudaMemcpyDeviceToHost` 
    * destination 포인터 : device 포인터
    * source 포인터 : host 포인터
* `cudaMemcpyHostToDevice`
    * destination 포인터 : host 포인터
    * source 포인터 : device 포인터   
* `cudaMemcpyDeviceToDevice`   
    * destination 포인터, source 포인터 : device 포인터   

※ source와 destination이 모두 host인 포인터일 경우, 표준 C의 memcpy()를 사용하면 된다.    



## 두번째 실습 : device 정보 가져오기


우리가 device를 활용하여 메모리를 할당하고 device 코드를 실행할때, device가 얼마나 많은 메모리를 가지고 성능이 어떤지를 알 수 있으면 유용할 것이라 생각 된다. 


먼저, 시스템에 CUDA 아키텍처 기반으로 제작된 디바이스가 몇개인지 알아보자.  

In [ ]:
 %%cu  
 //code 1.4
 #include <stdio.h> 
static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))
 int main(void) {
     int count;

     HANDLE_ERROR(cudaGetDeviceCount(&count));

     printf("device count : %d", count);

     return 0;
 }

device count : 1


우리는 위의 코드 1.4를 통해 디바이스의 갯수를 알 수 있었다.   
이제 우리는 각 device가 가진 정보들에 대해 알아보자.  

CUDA 런타임은 cudaDeviceProp 타입의 구조체를 통해 속성들을 반환한다.  
어떤 종류의 속성들이 포함되 있을 까?   

| Device 속성 | 설명 |
|:--------|:--------|
|  **char** name[256]       |  device를 식별해주는 아스키 문자열 (ex) GeForce GTX 280      |   
| **size\_t** totalGlobalMem | 바이트 단위의 device 전역 메모리의 양|    
| **size\_t** sharedMemPerBlock | 블록 당 이용할 수 있는 공유 메모리의 최대 양 (바이트 단위) |
| **int** regsPerBlock | 블록 당 이용할 수 있는 32비트 레지스터의 개수 |
| **int** warpSize | 하나의 warp가 갖는 스레드의 개수 |
| **size\_t** memPitch | 메모리 복사 시 허용되는 Pitch의 최대 크기(바이트 단위) |
| **int** maxThreadsPerBlock | 하나의 블록이 포함할 수 있는 스레드의 최대 개수 |
| **int** maxThreadsDim\[3\] | 하나의 블록에서 각 차원이 가질 수 있는 스레드의 최대 개수|    
| **int** maxGridSize\[3\] | 하나의 gird에서 각 차원이 가질 수 있는 블록의 최대 |    
| **size\_t** totalConstMem | 이용가능한 상수 메모리의 크기 |    
| **int** major | device 계산 능력의 주 개정 번호|   
| **int** minor | device 계산 능력의 부 개정 번호|   
| **size\_t** textureAlignment | 텍스처 정렬에 대한 device의 요구 사항|    
| **int** deviceOverlap | 이 device가 cudaMemcpy()와 커널을 동시에 수행할 수 있는 지를 나타내는 boolean 값|   
| **int** multiProcessorCount | device의 프로세서 개수 |    
| **int** kernelExecTimeoutEnabled |device에서 실행되는 커널에 런타임의 제한이 잇는 지를 나타내는 boolean 값|   
| **int** integrated | device가 통합형 GPU인지를 나타내는 boolean 값 |   
| **int** canMapHostMemory | device가 host  메모리를 CUDA device 주소 공간에 mapping가능한지를 나타내는 boolean 값 |    
| **int** computMode | device의 계산 모드를 나타내는 값 : default. exclusive(전용), prohibited(금지) |   
| **int** maxTexture1D | 지원하는 1D 텍스쳐의 크기 |   
| **int** maxTexture2D\[2\] | 2D 텍스처에서 지원하는 최대 차원의 개수|   
| **int** maxTexture3D\[3\] | 3D 텍스처에서 지원하는 최대 차원의 개수|   
| **int** maxTexture1DLayered\[2\] | 1D 레이어드 텍스처의 최대 차원의 개수|   
| **int** maxTexture2DLayered\[3\] | 2D 레이어드 텍스처의 최대 차원의 개수|   
| **int** concurrentKernel |  device가 같은 컨텍스트 내에서 다수의 커널을 동시에 실행할 수 있는 지 나타내는 boolean 값|   

이외에도 여러 중요한 속성들이 존재한다. 그들에 대해서는 [NVIDIA CUDA Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)를 참고하도록 하자. 


In [ ]:
%%cu   
 //code 1.5
 #include <stdio.h> 
static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))    
int main (void) {
    cudaDeviceProp prop;
    int count;

    HANDLE_ERROR( cudaGetDeviceCount( &count));

    for(int i = 0; i < count; i++) {
        HANDLE_ERROR( cudaGetDeviceProperties(&prop, i));
        printf(" --- General Information for device %d ---\n", i);
        printf("Name : %s \n", prop.name);
        printf("Comput capability : %d.%d\n", prop.major, prop.minor);
        printf("Clock rate : %d\n", prop.clockRate);
        printf("Device copy overlap: ");
        if (prop.deviceOverlap) {
            printf("Enabled\n");
        }
        else {
            printf("Disabled\n");
        }
        printf("Kernel execition timeout : ");
        if (prop.kernelExecTimeoutEnabled) {
            printf("Enabled\n");
        }
        else {
            printf("Disabled\n");
        }

        printf(" --- Memory Information for device %d---\n", i);
        printf("Total global mem : %ld\n", prop.totalGlobalMem);
        printf("Total constant Mem : %ld\n", prop.totalConstMem);
        printf("Max mem pitch : %ld\n", prop.memPitch);
        printf("Texture Alignment : %ld\n", prop.textureAlignment);
        printf(" --- MP Information for device %d ---\n", i);
        printf("Multiprocessor count : %d\n", prop.multiProcessorCount);
        printf("Shared mem per mp : %d\n", prop.sharedMemPerBlock);
        printf("Registers per mp : %d\n", prop.regsPerBlock);
        printf("Threads in warp : %d\n", prop.warpSize);
        printf("Max threads per block : %d\n", prop.maxThreadsPerBlock);
        printf("Max thread dimensions : (%d, %d, %d)\n",
               prop.maxThreadsDim[0],
               prop.maxThreadsDim[1],
               prop.maxThreadsDim[2]);
        printf("Max grid dimentsions : (%d, %d, %d)\n",
               prop.maxGridSize[0],
               prop.maxGridSize[1],
               prop.maxGridSize[2]);
        printf("\n");
    }
}

 --- General Information for device 0 ---
Name : Tesla P100-PCIE-16GB 
Comput capability : 6.0
Clock rate : 1328500
Device copy overlap: Enabled
Kernel execition timeout : Disabled
 --- Memory Information for device 0---
Total global mem : 17071734784
Total constant Mem : 65536
Max mem pitch : 2147483647
Texture Alignment : 512
 --- MP Information for device 0 ---
Multiprocessor count : 56
Shared mem per mp : 49152
Registers per mp : 65536
Threads in warp : 32
Max threads per block : 1024
Max thread dimensions : (1024, 1024, 64)
Max grid dimentsions : (2147483647, 65535, 65535)




## 세번째 실습 : device 속성 이용하기  
앞선 실습에서 우리는 CUDA 그래픽 카드의 세부사항에 대해 알아보았다.   
그런데 이러한 device의 속성을 어떻게 이용할 수 있을까?

우리는 프로그램이 실행될때, 해당 상황에서 가장 우수한 GPU를 선택하여 code를 실행하고자 할것이다.   
이를 통해 혹은 만약 커널이 CPU와 가까운 위치에서 서로 상호작용할 필요가 있을 때 우리는 CPU와 시스템 메모리를 공유하는 통합형 GPU에서 코드를 실행하고자 할것이다.  

이러한 속성들을 질의 하기 위해 `cudaGetDeviceProperties`를 이용해보자.   

예를 들어 특정 버전 이상의 CUDA 계산 능력을 보유한 카드가 지원되야하는 코드를 실행시킬때에는 이러한 조건을 만족하는 device를 최소한 한 개는 찾아야한다.   

여기서 code를 작성하여 예를 들기 위해 주버전이 1보다 크거나, 주버전이 1이고 부버전이 3이상인 device가 있는 검색해서 선택해보자.

In [ ]:
%%cu   
 //code 1.6
 #include <stdio.h> 
static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))    
int main(void) {
    cudaDeviceProp prop;//해당 구조체를 통해 우리가 원하는 속성을 가진 device를 찾아보자.
    int dev;

    HANDLE_ERROR(cudaGetDevice(&dev));
    printf("ID of current CUDA devices : %d\n", dev);//현재 device의 id를 가져온다.


    memset(&prop, 0, sizeof(cudaDeviceProp)); //현재 prop객체는 host에서 사용할 것이다 현재 main함수는 host 코드이니 memset을 사용하는 것이 맞다.

    prop.major = 1; //주 버전은 1이상이어야한다.
    prop.minor = 3; //부 버전은 주버전이 1일때 3이상이어야한다.

    HANDLE_ERROR( cudaChooseDevice(&dev, &prop));//prop의 조건에 맞는 device의id를 반환한다.
    //해당 조건을 만족하는 device가 여러개라면 해당 조건에 가장 가까운 device id를 반환한다.
    //docs : https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1gf61f9ae0fe2d93b5b968756684a49460

    printf("ID of CUDA device closest to revision 1.3 : %d\n", dev);
    HANDLE_ERROR(cudaSetDevice(dev));
    //이제 앞서 cudaChooseDevice()를 통해 반환된 cuda device id의 device에서 작업이 수행될 것이다.

}

ID of current CUDA devices : 0
ID of CUDA device closest to revision 1.3 : 0

